# Useful variable gathering.
## Notes
First iteration is going to be a series of linked Lats and Longs. Don't worry too much about elevation, or make
some small if condition that might discourage going paths with too much elevation maybe.

Describe lat and longs for each point in the path.

## Import data
Import the data using CSV files.

In [1]:
import pandas as pd
df = pd.read_csv('carrauntoohil_data_set.csv')
# zip() method combines iterables into a single iterator. Output is a tuple.
points = list(zip(df['latitude'], df['longitude'], df['elevation']))

# Return the highest and lowest values from the set.
Using Pandas dataframes and the min() and max() methods to determine the lowest and highest values of the set.

In [2]:
# Return the highest and lowest values in the data set.
lowest = min(points, key=lambda x: x[2])
highest = max(points, key=lambda x: x[2])
print(lowest, highest)

(52.00236741, -9.72965549, 334.6512145996094) (51.99895007, -9.74338607, 1006.589660644531)


## Create a Graph from data points using the NetworkX Library. 

In [3]:
from scipy.spatial import KDTree
import networkx as nx

# Graph Creation
G = nx.Graph()
tree = KDTree([(lat, lon) for lat, lon, _ in points])

## Loop through the points array.
This allows the data to be iterated and stored in a series of points.
This allows assigning of indices based on neighbour values.
Adds edge weighting based on the distance of elevation between points.
There's a lot going on here, may take a few seconds to run.

In [4]:
for i, (lat, lon, elev) in enumerate(points):
    G.add_node(i, pos=(lat, lon), elevation=elev)
    distances, indices = tree.query((lat, lon), k=5)
    for j, dist in zip(indices, distances):
        if j != i:
            elev_diff = abs(elev - points[j][2])
            G.add_edge(i,j, weight=elev_diff + dist)

### Define the source and destination points and use Dijkstra's(Shortest Path First) to find shortest path.
Using the Networkx module, changing the algorithm for pathfinding is as simple as changing the name of the nx method.

#### Using Shortest Path First Algorithm
`path = nx.shortest_path(G, source=start, target=end, weight="weight")` - Shortest path using shortest path algorithm.

#### Using A* Algorithm
`path = nx.astar_path(G, source=start, target=end, weight="weight")` - Shortest path using A*.

In [5]:
start = points.index(lowest)
end = points.index(highest)
path = nx.shortest_path(G, source=start, target=end, weight="weight")
path_coords = [(df.iloc[i]['latitude'], df.iloc[i]['longitude']) for i in path]
print(path_coords)

[(np.float64(52.00236741), np.float64(-9.72965549)), (np.float64(52.00236741), np.float64(-9.72980156)), (np.float64(52.00227748), np.float64(-9.72980156)), (np.float64(52.00218755), np.float64(-9.72980156)), (np.float64(52.00209762), np.float64(-9.72980156)), (np.float64(52.00200769), np.float64(-9.72980156)), (np.float64(52.00191776), np.float64(-9.72980156)), (np.float64(52.00182783), np.float64(-9.72980156)), (np.float64(52.00173789999999), np.float64(-9.72980156)), (np.float64(52.00173789999999), np.float64(-9.72994763)), (np.float64(52.00173789999999), np.float64(-9.7300937)), (np.float64(52.00173789999999), np.float64(-9.73023977)), (np.float64(52.00173789999999), np.float64(-9.73038584)), (np.float64(52.00164797), np.float64(-9.73038584)), (np.float64(52.00164797), np.float64(-9.73053191)), (np.float64(52.00155804), np.float64(-9.73053191)), (np.float64(52.00146811), np.float64(-9.73053191)), (np.float64(52.00146811), np.float64(-9.73067798)), (np.float64(52.00137818), np.float

# Using Folium to display the completed map.

In [ ]:
import folium
# Center the map at the midpoint between lowest and highest
mid_lat = (lowest[0] + highest[0]) / 2
mid_lon = (lowest[1] + highest[1]) / 2
m = folium.Map(location=[mid_lat, mid_lon], zoom_start=13)

# Add markers for the lowest and highest points
folium.Marker(location=[lowest[0], lowest[1]], 
              popup="Lowest Point", 
              icon=folium.Icon(color="green")).add_to(m)
folium.Marker(location=[highest[0], highest[1]], 
              popup="Highest Point", 
              icon=folium.Icon(color="red")).add_to(m)

# Add the path as a polyline
folium.PolyLine(locations=path_coords, color="blue", weight=3, opacity=0.7).add_to(m)

# Save map to HTML file
m.save("path_map.html")
print("Map has been saved to 'path_map.html'.")